In [2]:
"""
--------------
folder_sync.py
--------------

This module is used to initiate synchronization between two separate folders.
It uses the Windows 'xcopy' command to perform the sync, so the module
must be used in a Windows environment to work properly.

Initially, it was coded specifically to perform file sync between the nSpec 
tool local hard drive and the nSpec shared LAN archive folder.
This version has been generalized and can receive user arguments.
If ran as a script, the default behavior is to perform the nSpec folder
synchronization. Arguments can be passed if imported as a module.

Users provide a list of paired directories representing source and destination
folders. Any new/modified files in the source folders will be copied to the 
corresponding destination folders.

The sync is one-way only, meaning any changes made directly to the destination
folder will not be reflected in the local hard drive.
This allows the destination folder to contain all data ever recorded,
even when the source folder has been purged.
"""

import numpy as np
import sqlite3
import os
import math
import argparse
import shutil
import warnings
import time
import subprocess
import matplotlib.pyplot as plt
import seaborn as sns
import traceback
import pandas as pd
import sqlalchemy as sa
from sqlalchemy import text
from sqlalchemy import create_engine
import subprocess
import os
import time
from datetime import datetime
import re


class FolderSync:
    
    def __init__(self, folderpaths, log_path, skip_verify=False):
        """Initialize the filepaths variables.
        
        Args:
            filepaths (list): List of paired folderpaths. Expects an input
                in the form of '[..., [[src, dest]], ...]', where each
                'src' and 'dest' are the source and destination folder
                paths, respectively, and are strings.
            log_path (str): Represents the folder to save a log file into
                recording all files which were synced.
            skip_verify (bool, optional): If set to to True, the program
                will not ask for user verification via keyboard input
                before the filesync is initiated. The default is False.

        Returns:
            None.
        """
        self.list_of_dir = folderpaths
        self.log_path = log_path
        self.skip_verify = skip_verify
        
    def perform_sync(self, transfer=False, sql_query="", dest_db="", table_name=""):
        """Performs the file sync between the source and destination folders.

        Returns:
            None.
        """
        
        if not self.skip_verify:
            print("Please review the source and destination folders...\n")
            for idx, dir in enumerate(self.list_of_dir):
                print("SOURCE " + str(idx + 1) + ": " + dir[0])
                print("DESTINATION " + str(idx + 1) + ": " + dir[1])
            usrcon = input("\nType 'Yes' and press Enter "
                           + "to confirm initiation of folder sync: ")
            if usrcon == "Yes":
                print("\nProceeding with the folder sync...\n")
            else:
                print("\nAborting the folder sync!")
                return
            
        start = time.perf_counter()  # Track total time elapsed
        
        # Create a text file to capture log output.
        output_file = open(self.log_path + "placeholder.txt", "w")
        
        # Iterate through each source and destination.
        for dir in self.list_of_dir:
        
            print("CURRENT SOURCE: " + dir[0])
            print("CURRENT DESTINATION: " + dir[1])
        
            # Create the sync process using the xcopy command.
            process = subprocess.Popen(
                ["xcopy", dir[0], dir[1], "/f", "/e", "/d", "/c", "/y"], 
                stdout=subprocess.PIPE, stderr=subprocess.PIPE,
                shell = True, text = True)
        
            # List current source and destination in log file.
            output_file.write("CURRENT SOURCE: " + dir[0] + "\n")
            output_file.write("CURRENT DESTINATION: " + dir[1] + "\n")
  
            # Stream the sync output in real-time.
            for line in process.stdout:
                print(line, end="")
                output_file.write(line)  # Write each line to text file
                if transfer == True and line[-4:-1] == ".db":
                    mod_db = re.search(r"->\s(.*)$", line[0:-1]).group(1)
                    fetched_data = DbInterface(mod_db).fetch_from(sql_query)
                    DbInterface(dest_db).append_to(fetched_data, table_name)
                    
            output_file.write("\n")
            print("\n")
        
        end = time.perf_counter()  # Track total time elapsed
        
        print(f"Elapsed time: {end - start:.6f} seconds")
        output_file.write(f"Elapsed time: {end - start:.6f} seconds\n")
        
        # Get the date and time when sync was performed.
        current_datetime = ((datetime.now()).strftime("%Y-%m-%d %H:%M:%S"))
        rename_datetime = current_datetime.replace(" ", "_").replace(":", "-")
        output_file.write("Sync finished --> " + current_datetime + "\n")
        
        # Rename text file to current date and time.
        output_file.close()
        rename_log = self.log_path + str(rename_datetime) + ".txt"
        os.replace(self.log_path + "placeholder.txt", rename_log)
        
        print("Results log saved to " + rename_log)
        
        input("Press Enter to exit...")
        
if __name__ == "__main__":
    # List of all source and destination paths to perform copying.
    # This is specifically for the nSpec local folders -> LAN folders.
    folderpaths = [
        ["C:\\Users\\JChristensen01\\Downloads\\source\\",
         "C:\\Users\\JChristensen01\\Downloads\\dest\\"]]
    # The location for saving the log file.
    log_path = "C:\\Users\\JChristensen01\\Downloads\\dest\\"
    sql_query = text("""
    SELECT vwDefects.DefectID, 
           vwDefects.ImageID, 
           vwDefects.AnalysisID, 
           vwDefects.DeviceID, 
           vwDefects.X, 
           vwDefects.Y, 
           vwDefects.W, 
           vwDefects.H, 
           vwDefects.Area, 
           vwDefects.Intensity, 
           vwDefects.IntensityDeviation, 
           vwDefects.Eccentricity, 
           vwDefects.Orientation, 
           vwDefects.XinDevice, 
           vwDefects.YinDevice, 
           vwDefects.ClassID, 
           vwDefects.Score, 
           vwDefects.Contour, 
           vwDefects.ClassName, 
           ParameterGroups.Name, 
           SPA.PropertyData AS SampleID,
           SPB.PropertyData AS JobName
       
      FROM vwDefects 
           INNER JOIN Analysis 
           ON vwDefects.AnalysisID = Analysis.AnalysisID 
       
           INNER JOIN ScanProperties SPA 
           ON Analysis.ScanID = SPA.ScanID 
       
           INNER JOIN ScanProperties SPB 
           ON Analysis.ScanID = SPB.ScanID
       
           INNER JOIN ParameterGroups 
           ON Analysis.ParameterGroupGUID = ParameterGroups.GroupGUID 
       
     WHERE ParameterGroups.Name = '5XBF_BareWaferPRR' 
       AND SPA.PropertyName = 'SampleID'
       AND SPB.PropertyName = 'JobName'
    """)
    dest_db = "\\\\cam-vpnap-nas1\\nSpec\\Custom Databases\\prr_defect_statistics.db"
    table_name = "PRR_DEFECTS"
    sync_inst = FolderSync(folderpaths, log_path, True)
    sync_inst.perform_sync(True, sql_query, dest_db, table_name)

CURRENT SOURCE: C:\Users\JChristensen01\Downloads\source\
CURRENT DESTINATION: C:\Users\JChristensen01\Downloads\dest\
C:\Users\JChristensen01\Downloads\source\6-1189.db -> C:\Users\JChristensen01\Downloads\dest\6-1189.db
Database file not in correct format, returning empty dataframe
Successfully appended data to database connection
C:\Users\JChristensen01\Downloads\source\6-1231.db -> C:\Users\JChristensen01\Downloads\dest\6-1231.db
Data succesfully fetched
Successfully appended data to database connection
C:\Users\JChristensen01\Downloads\source\6-1232.db -> C:\Users\JChristensen01\Downloads\dest\6-1232.db
Data succesfully fetched
Successfully appended data to database connection
C:\Users\JChristensen01\Downloads\source\6-1233.db -> C:\Users\JChristensen01\Downloads\dest\6-1233.db
Data succesfully fetched
Successfully appended data to database connection
C:\Users\JChristensen01\Downloads\source\6-1234.db -> C:\Users\JChristensen01\Downloads\dest\6-1234.db
Data succesfully fetched
Suc

Press Enter to exit... 


In [1]:
class DbInterface:
    def __init__(self, filepath):
        self.filepath = filepath
    def fetch_from(self, query):
        engine = create_engine('sqlite:///' + self.filepath.replace('\\', '/').rstrip())
        conn = engine.connect()
        try:
            df_fetch = pd.read_sql(query, conn)
            if len(df_fetch) > 0:
                print("Data succesfully fetched")
                return df_fetch
            else:
                print("Database file returns empty list from query")
                return df_fetch
        except:
            print('Database file not in correct format, returning empty dataframe')
            return pd.DataFrame()

        conn.invalidate()
        engine.dispose()

    def append_to(self, df, table_name):
        engine = create_engine('sqlite:///' + self.filepath.replace('\\', '/').rstrip())
        conn = engine.connect()
        try:
            df.to_sql(table_name, con=conn, if_exists='append', index=False)
            print("Successfully appended data to database connection")
        except Exception as e:
            print(f"Data appending failed with error: {e}")

        conn.invalidate()
        engine.dispose()